## Dependencias

In [3]:
DIM_SIZE_REDUCTION = (1,1,1)
TARGET_MODE = 'target'
train_on_gpu = torch.cuda.is_available()

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

CUDA is available!  Training on GPU ...


In [ ]:
import dataset import get_train_valid_dataloaders
from dataset import PATHS
from train import train
from models import initialize_model

train_loader, valid_loader = get_train_valid_dataloaders(dim_size_reduction = DIM_SIZE_REDUCTION, target_mode = TARGET_MODE, train_path = PATHS["LQ_TRAIN"], valid_path = PATHS["LQ_VALID"])
model = initialize_model(train_on_gpu=True, arch = "UNet3D")
train(model, train_loader, valid_loader, model_name=MODEL_NAME, n_epochs=100, loss_function="dice", AMP=True, train_on_gpu=train_on_gpu)